# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [13]:
using Pkg
Pkg.add("MLDataUtils")
Pkg.add("GLM")
using GLM, RDatasets, MLDataUtils

  Resolving package versions...
   Updating `C:\Users\s7070\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\s7070\.julia\environments\v1.4\Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `C:\Users\s7070\.julia\environments\v1.4\Project.toml`
 [no changes]
   Updating `C:\Users\s7070\.julia\environments\v1.4\Manifest.toml`
 [no changes]


## 讀取資料

In [8]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [15]:
#轉換資料
boston[!,:LogMedV] = log.(boston[!, :MedV])

506-element Array{Float64,1}:
 3.1780538303479458
 3.0726933146901194
 3.5467396869528134
 3.5085558999826545
 3.5890591188317256
 3.3568971227655755
 3.131136910560194
 3.299533727885655
 2.803360380906535
 2.9391619220655967
 2.70805020110221
 2.9391619220655967
 3.077312260546414
 ⋮
 3.1986731175506815
 3.139832617527748
 2.9806186357439426
 2.9069010598473755
 3.054001181677967
 2.8622008809294686
 2.8213788864092133
 3.109060958860994
 3.0252910757955354
 3.173878458937465
 3.091042453358316
 2.4765384001174837

In [20]:
#訓練與測試資料集
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [21]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.11747,12.5,7.87,0,0.524,6.009,82.9,6.2267,5,311
2,15.0234,0.0,18.1,0,0.614,5.304,97.3,2.1007,24,666
3,0.05646,0.0,12.83,0,0.437,6.232,53.7,5.0141,5,398
4,0.24522,0.0,9.9,0,0.544,5.782,71.7,4.0317,4,304
5,0.03705,20.0,3.33,0,0.4429,6.968,37.2,5.2447,5,216
6,1.25179,0.0,8.14,0,0.538,5.57,98.1,3.7979,4,307
7,0.34109,0.0,7.38,0,0.493,6.415,40.1,4.7211,5,287
8,1.23247,0.0,8.14,0,0.538,6.142,91.7,3.9769,4,307
9,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222


In [22]:
#使用線性迴歸模型
model = lm(@formula(LogMedV ~ Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

LogMedV ~ 1 + Crim + Chas + NOx + Rm + Dis + Rad + Tax + PTRatio + Black + LStat

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                 Estimate   Std. Error    t value  Pr(>|t|)     Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)   4.30951      0.233032      18.4932     <1e-54   3.85137       4.76765
Crim         -0.0110484    0.00151333    -7.30075    <1e-11  -0.0140236    -0.00807322
Chas          0.103835     0.0375335      2.76645    0.0059   0.0300437     0.177626
NOx          -0.835696     0.164455      -5.08162    <1e-6   -1.15901      -0.512378
Rm            0.0812661    0.0178325      4.5572     <1e-5    0.0462075     0.116325
Dis          -0.0443144    0.00733629    -6.

In [23]:
#預測
exp.(predict(model,test))

101-element Array{Float64,1}:
 21.919422691543147
 11.960746404583979
 21.156529933660764
 18.552729783519712
 36.554957562297
 13.97660421884797
 25.61815338039068
 15.662214238759809
 26.843394960658586
 24.970489550984247
 13.177422491248228
 17.667242276383455
 25.79239356312155
  ⋮
 31.513450433187536
 23.616551894720587
 11.491662467927007
 29.004827423900625
 15.038784465866428
 35.013589817955605
 18.327856866968563
 22.357483791607557
 14.199585778213745
 22.36222002438967
 19.045867365533997
 16.75869368382322

In [24]:
#評估模型
adjr2(model)

0.7884305850248672